In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

from ngboost import NGBRegressor
from scoringrules import crps_ensemble
from ngboost.distns import Normal
from ngboost.scores import CRPScore


import numpy as np
from scipy.stats import norm

# 1) Load
X = pd.read_csv("Data/X_trn.csv")
y = pd.read_csv("Data/Y_trn.csv").squeeze("columns").astype(float).to_numpy().ravel()

# 2) Feature groups
num_cols = ["year", "age", "prestg10", "childs"]
cat_cols = ["occrecode", "wrkstat", "gender", "educcat", "maritalcat"]

# (optional) coerce numerics in case they were read as strings
for c in num_cols:
    X[c] = pd.to_numeric(X[c], errors="coerce")

# 3) Preprocess: numeric imputation + dense one-hot for categoricals
numeric = Pipeline([("imputer", SimpleImputer(strategy="median"))])
categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),  # <-- dense OHE
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric, num_cols),
        ("cat", categorical, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=0.0,  # <-- ensure the whole output is dense
)

# 4) Model: preprocess -> NGBoost (natural gradients off)
model = Pipeline([
    ("prep", preprocessor),  # your SimpleImputer + OneHotEncoder(sparse_output=False)
    ("ngb", NGBRegressor(
        #Dist=Normal,
        #Score=CRPScore,
        random_state=42,
        natural_gradient=False   # avoids the np.linalg.solve shape issue you hit earlier
    )),
])

# 5) Split & fit
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model.fit(X_train, y_train)

# 6) Metrics: MSE, NLL
prep = model.named_steps["prep"]
ngb  = model.named_steps["ngb"]
X_test_tx = prep.transform(X_test)  # dense ndarray

y_point = ngb.predict(X_test_tx)
y_dist  = ngb.pred_dist(X_test_tx)

mse = mean_squared_error(y_test, y_point)
nll = -y_dist.logpdf(y_test).mean()
print("Test MSE:", float(mse))
print("Test NLL:", float(nll))

# 7) CRPS from 1000 predictive samples per row
n_samples = 100
samples = y_dist.sample(n_samples)
if hasattr(samples, "detach"):  # torch -> numpy if used
    samples = samples.detach().cpu().numpy()
samples = np.asarray(samples)

# normalize shape to (n_obs, n_samples)
if samples.shape == (n_samples, len(y_test)):
    samples = samples.T
elif samples.shape != (len(y_test), n_samples):
    # fallback if .sample not present/behaves differently: assume Normal loc/scale
    mu = np.asarray(getattr(y_dist, "loc"))
    sigma = np.asarray(getattr(y_dist, "scale"))
    rng = np.random.default_rng(42)
    samples = mu[:, None] + sigma[:, None] * rng.standard_normal((mu.shape[0], n_samples))

crps = crps_ensemble(y_test, samples).mean().item()
print("Test CRPS:", crps)

print("Score class:", ngb.Score.__name__)
print("Dist class :", ngb.Dist.__name__)